<a href="https://colab.research.google.com/github/Mathias0210/Bach/blob/main/Pyth2fors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
!pip install mlxtend

In [2]:
from google.colab import files
uploaded = files.upload()
df = pd.read_excel('SalesData.xlsx')
# show the first few rows
df.head()

Saving SalesData.xlsx to SalesData (1).xlsx


,TransaktionsID,RækkeID,KundeID,Navn,Adresse1,Postnr,LevBynavn,Land,ExtSystemId,KategoriID,ProduktID,ProductName,Uom,Unnamed: 13,Unnamed: 14
0,21221405,75277762,996409,Hansens VVS APS,NaN,3230.0,Græsted,DK,lemu-dk,Metro Therm,345166400,Metro Therm el-vandvarmer model 30 type 907 26...,STK,NaN,NaN
1,21229498,75280255,995782,Krøll Cranes A/S,NaN,3540.0,Lynge,DK,lemu-dk,Slange PA,7830309398,Slange PA6 SW-Ø21 sort 50M,RUL,NaN,KundeID
2,21121214,75280118,995782,Krøll Cranes A/S,NaN,3540.0,Lynge,DK,lemu-dk,C Karakteristik,0422207630,Automatsikring 6KA 3P C 10A 5SY6310-7,STK,NaN,996409
3,21072615,75277996,995782,Krøll Cranes A/S,NaN,3540.0,Lynge,DK,lemu-dk,Ligekærv LK,4323400050,"Wiha skruetrækker slimFix LK 3,5 VDE 1000V",STK,NaN,995782
4,21072615,75278024,995782,Krøll Cranes A/S,NaN,3540.0,Lynge,DK,lemu-dk,CEE stikpropper 63 ampere,1515134033,CEE stikprop 5 polet 63A 400V IP44,STK,NaN,995769


In [3]:
# Prepend '_' to ProduktID
df['ProduktID'] = df['ProduktID'].apply(lambda x: '_'+str(x))

#Sætter alle produktnavne til stort
df['ProductName']=df['ProductName'].str.upper()

In [4]:
# DataFrame for building the recommendation system
orders = df[['TransaktionsID', 'ProduktID']]
print(orders.shape)
print(orders.head(10))
# DataFrame for retrieving product descriptions
products = df[['ProduktID', 'ProductName']].copy()

print(products.shape)
print(products.head(10))

(10000, 2)
   TransaktionsID    ProduktID
0        21221405   _345166400
1        21229498  _7830309398
2        21121214  _0422207630
3        21072615  _4323400050
4        21072615  _1515134033
5        21072615  _1515132035
6        21222356  _4443160332
7        21222356  _4326906029
8        21218223  _2119002052
9        21218223  _2119808667
(10000, 2)
     ProduktID                                        ProductName
0   _345166400  METRO THERM EL-VANDVARMER MODEL 30 TYPE 907 26...
1  _7830309398                         SLANGE PA6 SW-Ø21 SORT 50M
2  _0422207630              AUTOMATSIKRING 6KA 3P C 10A 5SY6310-7
3  _4323400050         WIHA SKRUETRÆKKER SLIMFIX LK 3,5 VDE 1000V
4  _1515134033                 CEE STIKPROP 5 POLET 63A 400V IP44
5  _1515132035             CEE FORLÆNGERLED 5 POLET 63A 400V IP44
6  _4443160332             BOREPATRON TIL M18 FPD/FDD, M18 ONEPD,
7  _4326906029                 TAJIMA 5M HI LOCK 25MM RØD KPS KL1
8  _2119002052  SIKKERHEDSAFBRYDER 16A 3 PO

In [5]:
# Drop duplicated products
products = products[~products.duplicated()]


print(products.shape)
print(products.head(10))

(6477, 2)
     ProduktID                                        ProductName
0   _345166400  METRO THERM EL-VANDVARMER MODEL 30 TYPE 907 26...
1  _7830309398                         SLANGE PA6 SW-Ø21 SORT 50M
2  _0422207630              AUTOMATSIKRING 6KA 3P C 10A 5SY6310-7
3  _4323400050         WIHA SKRUETRÆKKER SLIMFIX LK 3,5 VDE 1000V
4  _1515134033                 CEE STIKPROP 5 POLET 63A 400V IP44
5  _1515132035             CEE FORLÆNGERLED 5 POLET 63A 400V IP44
6  _4443160332             BOREPATRON TIL M18 FPD/FDD, M18 ONEPD,
7  _4326906029                 TAJIMA 5M HI LOCK 25MM RØD KPS KL1
8  _2119002052  SIKKERHEDSAFBRYDER 16A 3 POLET  MED HJÆLPEKONT...
9  _2119808667  HJÆLPEKONTAKT 1 SLUTTE FOR 16/125A AFBRYDER OA...


In [6]:
# Fra tidligere trin er der nu kun ca. 6500 linjer i produkt men det bemærkes 

#products.to_excel('Products.xlsx')
#files.download('Products.xlsx')

#Det bemærkes at forekllige productnavne har samme produktID, ved nærmere inspektion er det dog de samme produkter bare med forskellige navne.

In [7]:
#Fjerner duplikerede linjer med productID og beholder


products = (products
.sort_values(by=['ProduktID'],ascending=False))

products.head(160)

,ProduktID,ProductName
6964,_RIVS810,"RUSTFRIE STÅLRØR EN 1.4301 GLØDEDE 8X1,00 MM"
5611,_RIVG4832,"RUSTFRIE GEVINDRØR EN 1.4301 GLØDEDE 48,3X3,25 MM"
9581,_RIV4603,"RUSTFRIE STÅLRØR EN 1.4301 60X60X3,00 MM"
858,_RIV4404,"RUSTFRIE STÅLRØR EN 1.4301 40X40X4,00 MM"
8118,_RIV4403,"RUSTFRIE STÅLRØR EN 1.4301 40X40X3,00 MM"
...,...,...
1472,_9978864311,VENTILATOR EDM 100 HZ
9679,_9978863079,SILENT 200 CZ
9044,_9978858000,"THERMEX SILENT ONE, HVID"
3943,_9978858000,THERMEX SILENT ONE Ø100 HVID


In [8]:
#Fjerner fejlene fra ProdutData ved at overskrive dubletterne

products = products.drop_duplicates(subset='ProduktID', keep="last")
print(products.shape)
products.head(160)

(6351, 2)


,ProduktID,ProductName
6964,_RIVS810,"RUSTFRIE STÅLRØR EN 1.4301 GLØDEDE 8X1,00 MM"
5611,_RIVG4832,"RUSTFRIE GEVINDRØR EN 1.4301 GLØDEDE 48,3X3,25 MM"
9581,_RIV4603,"RUSTFRIE STÅLRØR EN 1.4301 60X60X3,00 MM"
858,_RIV4404,"RUSTFRIE STÅLRØR EN 1.4301 40X40X4,00 MM"
8118,_RIV4403,"RUSTFRIE STÅLRØR EN 1.4301 40X40X3,00 MM"
...,...,...
1472,_9978864311,VENTILATOR EDM 100 HZ
9679,_9978863079,SILENT 200 CZ
3943,_9978858000,THERMEX SILENT ONE Ø100 HVID
2629,_9978857386,SILENT 100 CHRZ - DESIGN


In [9]:
# Set the index to StockCode
products = products.set_index('ProduktID')

# Convert to Series for eve easier lookups
products = products['ProductName']

In [10]:
#Der blevv fjernet lidt over 100 fejl med denne metode og jeg må antage at produkterne er de samme og tabellen nu er klar til brug


#Jeg går nu videre til min liste af transaktioner
#Et loop der laver en liste med alle produktid'er for hver transaktion

def string_list(x):
    return [str(i) for i in x]

orders = orders.groupby('TransaktionsID')['ProduktID'].apply(list).reset_index()

#Der var sneget sig et par ekstra [] ind i produktname som jeg her fjerner

df['ProduktID'] = df['ProduktID'].str.strip('[]')
orders.head(100)


,TransaktionsID,ProduktID
0,16458869,[_1016008150]
1,16671499,[_3980020020]
2,16686990,"[_4419003986, _4438001708, _728428023]"
3,17632461,[_980419066]
4,18029570,[_7818230573]
...,...,...
95,21051238,[_7897992599]
96,21052330,[_7897700998]
97,21052385,"[_4333956718, _4333956890, _4333956719, _43299..."
98,21053098,"[_4361904104, _5788180022, _RIVS810]"


In [11]:
#orders.to_excel('orders.xlsx')
#files.download('orders.xlsx')

In [12]:
#Importerer mlxtend https://towardsdatascience.com/mlxtend-a-python-library-with-interesting-tools-for-data-science-tasks-d54c723f89cd

from mlxtend.preprocessing import TransactionEncoder

In [13]:
# fit the TransactionEncoder
te = TransactionEncoder()

te.fit(orders['ProduktID'])

# do the transformation
orders_1hot = te.transform(orders['ProduktID'])

#Omdanner til en matrix med produktID som kolonner og transaktioner som rækker.
orders_1hot = pd.DataFrame(orders_1hot, columns =te.columns_)
orders_1hot.head()

,_000090104,_000090106,_000090108,_000090110,_000090126,_000090404,_000090406,_000092104,_000092106,_000092108,...,_RGKF303,_RIG14832,_RIHF3320,_RIV4253,_RIV4303,_RIV4403,_RIV4404,_RIV4603,_RIVG4832,_RIVS810
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
#Importerer apriori som er vorres unsupervised metode
from mlxtend.frequent_patterns import apriori

#Instantierer apriori med datasettet der er klargjort og viser første associationer

ap = apriori(orders_1hot, min_support=0.01, max_len=5, use_colnames=True)
ap.head(6)

,support,itemsets
0,0.011673,(_1017036338)
1,0.010376,(_1017054433)
2,0.010052,(_1017056266)
3,0.011349,(_1017056279)
4,0.010376,(_1017056402)


In [15]:
from mlxtend.frequent_patterns import association_rules

#Viser her de stærkeste associationer
rules = association_rules(ap,metric="support", min_threshold = 0.0001)
print(rules)

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []


In [16]:
#Funktion der laver foreslag baseret på produktID

def predict(antecedent, rules, max_results= 6):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results]
  

    preds = predict({'_1017036338'}, rules)
    preds

    # Display the descriptions of the predictions
    for produktID in preds:  
      print(products[produktID])  


      #Konklusion, der er for mange produkter, for få transaktioner og ikke stærke nok associationer til at Apriori kan give anbefalinger...